In [1]:
import numpy as np
import torch
import torch.nn as tnn
import torch.nn.functional as F
import torch.optim as topti
from torchtext import data
from torchtext.vocab import GloVe
from imdb_dataloader import IMDB

# 构建神经网络

In [44]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Class for creating the neural network.
class Network(tnn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.in_dropout = tnn.Dropout(0.15)
        self.lstm1 = tnn.LSTM(50, 300, 1,batch_first=True)
        self.selu = tnn.SELU()
        self.dropout = tnn.Dropout(0.5)
        self.dense1 = tnn.Linear(300, 200)
        self.norm1 = tnn.BatchNorm1d(200)
        self.dense2 = tnn.Linear(200, 128)
        self.dense3 = tnn.Linear(128,64)
        self.norm2 = tnn.BatchNorm1d(64)
        self.dense4 = tnn.Linear(64,1)

    def forward(self, input, length):
        """
        DO NOT MODIFY FUNCTION SIGNATURE
        Create the forward pass through the network.
        """
        x = self.in_dropout(input)
        o,(h_n,h_c) = self.lstm1(x)#[0][-1].view(-1,300)
        x = self.selu(o[:,-1,:].view(-1,300))
        x = self.dense1(x)
        x = self.norm1(x)
        x += (torch.randn(x.shape[1])*0.15).to(device)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dropout(x)
        x = self.selu(x)
        x = self.dense3(x)
        x = self.norm2(x)
        x = self.dropout(x)
        x = self.dense4(x)

        
        return (x).view(-1)

In [3]:
class PreProcessing():

    def pre(x):
        
        """Called after tokenization 把句子拆成单词 """
        """
        GRAM too small ....
        """
#         _arr = np.array([item for sublist in x for item in sublist]).reshape(1,-1)
#         for i,dim in enumerate(_arr):
#             _index = np.argwhere((dim == '/><br') == True).flatten()
#             for j in _index:
#                 _arr[i][j] = ""
#                 try:
#                     _arr[i][j-1] =_arr[i][j-1].replace("<br")
#                     _arr[i][j+1] =_arr[i][j+1].replace("/>")
#                 except IndexError:
#                     print("outofbound")
#         #return _arr.flatten().tolist()
        return x
    def post(batch, vocab):
    
        return batch
    text_field = data.Field(lower=True, include_lengths=True, batch_first=True, preprocessing=pre, postprocessing=post)

In [4]:
def lossFunc():
    """
    Define a loss function appropriate for the above networks that will
    add a sigmoid to the output and calculate the binary cross-entropy.
    """
    return tnn.BCELoss()

In [5]:
def save(_net,PATH='./model.pth',device=torch.device('cuda')):
    _net.to(device)
    torch.save(_net.state_dict(),PATH)

In [6]:
def evaluate(_net):
    num_correct = 0
    with torch.no_grad():
        for batch in testLoader:
            # Get a batch and potentially send it to GPU memory.
            inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)

            labels -= 1

            # Get predictions
            outputs = torch.sigmoid(_net(inputs, length))
            predicted = torch.round(outputs)
            num_correct += torch.sum(labels == predicted).item()

    accuracy = 100 * num_correct / len(dev)
    print(f"Classification accuracy: {accuracy}")
    return accuracy

## 加载数据集

In [7]:


textField = PreProcessing.text_field
labelField = data.Field(sequential=False)

train, dev = IMDB.splits(textField, labelField, train="train", validation="dev")

textField.build_vocab(train, dev, vectors=GloVe(name="6B", dim=50))
labelField.build_vocab(train, dev)

trainLoader, testLoader = data.BucketIterator.splits((train, dev), shuffle=True, batch_size=64,
                                                     sort_key=lambda x: len(x.text), sort_within_batch=True)


In [45]:
def main():
    # Use a GPU if available, as it should be faster.
    print("Using device: " + str(device))

    # Load the training dataset, and create a data loader to generate a batch.

    net = Network().to(device)
    criterion =lossFunc()
    optimiser = topti.Adam(net.parameters(), lr=1e-3)  # Minimise the loss using the Adam algorithm.
#     scheduler = torch.optim.lr_scheduler.StepLR(optimiser,step_size=4, gamma=0.6)
#     scheduler = torch.optim.lr_scheduler.ExponentialLR(optimiser, 0.9, last_epoch=-1)
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimiser,10)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser,patience=5)
    for epoch in range(35):
        running_loss = 0
        for i, batch in enumerate(trainLoader):
            # Get a batch and potentially send it to GPU memory.
            inputs, length, labels = textField.vocab.vectors[batch.text[0]].to(device), batch.text[1].to(
                device), batch.label.type(torch.FloatTensor).to(device)
            
            labels -= 1            
            # PyTorch calculates gradients by accumulating contributions to them (useful for
            # RNNs).  Hence we must manually set them to zero before calculating them.
            optimiser.zero_grad()
            
            # Forward pass through the network.
            output = torch.sigmoid(net(inputs, length))

            loss = criterion(output, labels)

            # Calculate gradients.
            loss.backward()

            # Minimise the loss according to the gradient.
            optimiser.step()
            
            running_loss += loss.item()
        
            if i % 32 == 31:
                print("Epoch: %2d, Batch: %4d, Loss: %.3f" % (epoch + 1, i + 1, running_loss / 32))
                running_loss = 0
        scheduler.step(running_loss)
        
    # Evaluation and save model
        print()
        print("###########################################")
        print("Current Validation Acc")
        new_acc = evaluate(net)
        PATH = "./model.pth"
        old_net = Network().to(device)
        try:
            old_net.load_state_dict(torch.load(PATH))
        except OSError:
            print("No available model!")
        print("Former Validation Acc")
        old_acc = evaluate(old_net)
        if new_acc > old_acc:
            save(net)
            print()
            print("Better Accuracy!!!")
            print("Saved model")
        print("###########################################\n")
    print("Trainning Complete")
    
    # Make the best acc model for cpu
    net = Network().to(device)
    net.load_state_dict(torch.load(PATH))
    print("Best Result: ")
    acc = evaluate(net)
    save(net,device=torch.device('cpu'))
    
if __name__ == '__main__':
    main()

Using device: cuda:0
Epoch:  1, Batch:   32, Loss: 0.749
Epoch:  1, Batch:   64, Loss: 0.713
Epoch:  1, Batch:   96, Loss: 0.690
Epoch:  1, Batch:  128, Loss: 0.721
Epoch:  1, Batch:  160, Loss: 0.715
Epoch:  1, Batch:  192, Loss: 0.673
Epoch:  1, Batch:  224, Loss: 0.630
Epoch:  1, Batch:  256, Loss: 0.559
Epoch:  1, Batch:  288, Loss: 0.560
Epoch:  1, Batch:  320, Loss: 0.544
Epoch:  1, Batch:  352, Loss: 0.502
Epoch:  1, Batch:  384, Loss: 0.528

###########################################
Current Validation Acc
Classification accuracy: 75.32010243277848
No available model!
Former Validation Acc
Classification accuracy: 48.35147247119078

Better Accuracy!!!
Saved model
###########################################

Epoch:  2, Batch:   32, Loss: 0.473
Epoch:  2, Batch:   64, Loss: 0.513
Epoch:  2, Batch:   96, Loss: 0.485
Epoch:  2, Batch:  128, Loss: 0.491
Epoch:  2, Batch:  160, Loss: 0.469
Epoch:  2, Batch:  192, Loss: 0.477
Epoch:  2, Batch:  224, Loss: 0.475
Epoch:  2, Batch:  256

Epoch: 13, Batch:   64, Loss: 0.415
Epoch: 13, Batch:   96, Loss: 0.382
Epoch: 13, Batch:  128, Loss: 0.416
Epoch: 13, Batch:  160, Loss: 0.393
Epoch: 13, Batch:  192, Loss: 0.389
Epoch: 13, Batch:  224, Loss: 0.384
Epoch: 13, Batch:  256, Loss: 0.390
Epoch: 13, Batch:  288, Loss: 0.382
Epoch: 13, Batch:  320, Loss: 0.389
Epoch: 13, Batch:  352, Loss: 0.382
Epoch: 13, Batch:  384, Loss: 0.391

###########################################
Current Validation Acc
Classification accuracy: 81.59411011523687
Former Validation Acc
Classification accuracy: 81.37003841229193

Better Accuracy!!!
Saved model
###########################################

Epoch: 14, Batch:   32, Loss: 0.384
Epoch: 14, Batch:   64, Loss: 0.375
Epoch: 14, Batch:   96, Loss: 0.397
Epoch: 14, Batch:  128, Loss: 0.375
Epoch: 14, Batch:  160, Loss: 0.366
Epoch: 14, Batch:  192, Loss: 0.371
Epoch: 14, Batch:  224, Loss: 0.404
Epoch: 14, Batch:  256, Loss: 0.361
Epoch: 14, Batch:  288, Loss: 0.381
Epoch: 14, Batch:  320, Los

Epoch: 25, Batch:  192, Loss: 0.274
Epoch: 25, Batch:  224, Loss: 0.256
Epoch: 25, Batch:  256, Loss: 0.266
Epoch: 25, Batch:  288, Loss: 0.256
Epoch: 25, Batch:  320, Loss: 0.268
Epoch: 25, Batch:  352, Loss: 0.257
Epoch: 25, Batch:  384, Loss: 0.263

###########################################
Current Validation Acc
Classification accuracy: 84.28297055057618
Former Validation Acc
Classification accuracy: 84.89116517285531
###########################################

Epoch: 26, Batch:   32, Loss: 0.252
Epoch: 26, Batch:   64, Loss: 0.240
Epoch: 26, Batch:   96, Loss: 0.248
Epoch: 26, Batch:  128, Loss: 0.225
Epoch: 26, Batch:  160, Loss: 0.238
Epoch: 26, Batch:  192, Loss: 0.251
Epoch: 26, Batch:  224, Loss: 0.253
Epoch: 26, Batch:  256, Loss: 0.238
Epoch: 26, Batch:  288, Loss: 0.260
Epoch: 26, Batch:  320, Loss: 0.268
Epoch: 26, Batch:  352, Loss: 0.271
Epoch: 26, Batch:  384, Loss: 0.232

###########################################
Current Validation Acc
Classification accuracy: 84